#Task 2 - Trying Different Models

###Importing the libraries

In [2]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from tensorflow import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

###Importing the datasets (code used from the baseline respository)

In [1]:
#*********************Comment this out and change them appropriately if you wish to run this locally******************************#
path_ptbdb_1 = "/content/drive/MyDrive/ML4HC/archive/ptbdb_normal.csv"
path_ptbdb_2 = "/content/drive/MyDrive/ML4HC/archive/ptbdb_abnormal.csv"

path_mitbih_train = "/content/drive/MyDrive/ML4HC/archive/mitbih_train.csv"
path_mitbih_test = "/content/drive/MyDrive/ML4HC/archive/mitbih_test.csv"

In [4]:
#function needed to reshape input
def convert3Dto2D(X):
  return X.reshape(-1, X.shape[1])

######PTBDB dataset

In [5]:
df_1 = pd.read_csv(path_ptbdb_1, header=None)
df_2 = pd.read_csv(path_ptbdb_2, header=None)

df_ptbdb = pd.concat([df_1, df_2])

df_ptbdb_train, df_ptbdb_test = train_test_split(df_ptbdb, test_size=0.2, random_state=1337, stratify=df_ptbdb[187])

Y_ptbdb_train = np.array(df_ptbdb_train[187].values).astype(np.int8)
X_ptbdb_train = np.array(df_ptbdb_train[list(range(187))].values)[..., np.newaxis]
X_ptbdb_train = convert3Dto2D(X_ptbdb_train)

Y_ptbdb_test = np.array(df_ptbdb_test[187].values).astype(np.int8)
X_ptbdb_test = np.array(df_ptbdb_test[list(range(187))].values)[..., np.newaxis]
X_ptbdb_test = convert3Dto2D(X_ptbdb_test)

######mitbih dataset

In [6]:
df_mitbih_train = pd.read_csv(path_mitbih_train, header=None)
df_mitbih_train = df_mitbih_train.sample(frac=1)
df_mitbih_test = pd.read_csv(path_mitbih_test, header=None)

Y_mitbih_train = np.array(df_mitbih_train[187].values).astype(np.int8)
X_mitbih_train = np.array(df_mitbih_train[list(range(187))].values)[..., np.newaxis]
X_mitbih_train = convert3Dto2D(X_mitbih_train)

Y_mitbih_test = np.array(df_mitbih_test[187].values).astype(np.int8)
X_mitbih_test = np.array(df_mitbih_test[list(range(187))].values)[..., np.newaxis]
X_mitbih_test = convert3Dto2D(X_mitbih_test)

In [18]:
print(np.shape(X_mitbih_train))

(87554, 187)


In [19]:
print(np.shape(X_ptbdb_test))

(2911, 187)


###Create & Train separate models

######Random Forrest

In [7]:
rf_classifier_mitbih = RandomForestClassifier(n_estimators = 100, max_depth = 5,criterion = 'entropy', random_state = 0)
rf_classifier_mitbih.fit(X_mitbih_train, Y_mitbih_train)
rf_classifier_ptbdb = RandomForestClassifier(n_estimators = 100, max_depth = 5,criterion = 'entropy', random_state = 0)
rf_classifier_ptbdb.fit(X_ptbdb_train, Y_ptbdb_train)

RandomForestClassifier(criterion='entropy', max_depth=5, random_state=0)

#####XGBoost

In [8]:
#initial model
# xgb1 = XGBClassifier(learning_rate=0.1, n_estimators=800,
#                       max_depth=7,
#                       min_child_weight=2,
#                       gamma=0.1,
#                       subsample=0.8,
#                       colsample_bytree=0.8,
#                       nthread=4,
#                       num_class=4,
#                       seed=43)

xgb_classifier_mitbih = XGBClassifier()
xgb_classifier_ptbdb = XGBClassifier()
xgb_classifier_mitbih.fit(X_mitbih_train, Y_mitbih_train)
xgb_classifier_ptbdb.fit(X_ptbdb_train,Y_ptbdb_train)

XGBClassifier()

######LightGBM

In [9]:
lgb_classifier_mitbih = lgb.LGBMClassifier(learning_rate=0.1,max_depth=-5,random_state=0)
lgb_classifier_mitbih.fit(X_mitbih_train,Y_mitbih_train,eval_set=[(X_mitbih_test,Y_mitbih_test),(X_mitbih_train,Y_mitbih_train)],
          verbose=20,eval_metric='logloss')
lgb_classifier_ptbdb = lgb.LGBMClassifier(learning_rate=0.1,max_depth=-5,random_state=0)
lgb_classifier_ptbdb.fit(X_ptbdb_train,Y_ptbdb_train,eval_set=[(X_ptbdb_test,Y_ptbdb_test),(X_ptbdb_train,Y_ptbdb_train)],
          verbose=20,eval_metric='logloss')

[20]	training's multi_logloss: 0.141762	training's multi_logloss: 0.141762	valid_0's multi_logloss: 0.1667	valid_0's multi_logloss: 0.1667
[40]	training's multi_logloss: 0.0764924	training's multi_logloss: 0.0764924	valid_0's multi_logloss: 0.111476	valid_0's multi_logloss: 0.111476
[60]	training's multi_logloss: 0.0512005	training's multi_logloss: 0.0512005	valid_0's multi_logloss: 0.0926366	valid_0's multi_logloss: 0.0926366
[80]	training's multi_logloss: 0.038395	training's multi_logloss: 0.038395	valid_0's multi_logloss: 0.0841577	valid_0's multi_logloss: 0.0841577
[100]	training's multi_logloss: 0.0302692	training's multi_logloss: 0.0302692	valid_0's multi_logloss: 0.0795949	valid_0's multi_logloss: 0.0795949
[20]	training's binary_logloss: 0.18533	training's binary_logloss: 0.18533	valid_0's binary_logloss: 0.215227	valid_0's binary_logloss: 0.215227
[40]	training's binary_logloss: 0.0935972	training's binary_logloss: 0.0935972	valid_0's binary_logloss: 0.133151	valid_0's binary_

LGBMClassifier(max_depth=-5, random_state=0)

####Grid Search to find the best models for each model type

######Random Forrest

In [10]:
#parameters to search for in random forrest
parameters = [{'n_estimators': [100,200,400,500], 
               'criterion': ['gini', 'entropy'],
               'max_depth': [4,5,6],
               'random_state': [0,43]
               }]

In [ ]:
#mitbih dataset
grid_search_mitbih_rf = GridSearchCV(estimator = rf_classifier_mitbih,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_mitbih_rf.fit(X_mitbih_train, Y_mitbih_train)
best_accuracy_mitbih_rf = grid_search_mitbih_rf.best_score_
best_parameters_mitbih_rf = grid_search_mitbih_rf.best_params_
print("mitbih dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_mitbih_rf*100))
print("Best Parameters:", best_parameters_mitbih_rf)

In [ ]:
#ptbdb dataset
grid_search_ptbdb_rf = GridSearchCV(estimator = rf_classifier_ptbdb,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_ptbdb_rf.fit(X_ptbdb_train, Y_ptbdb_train)
best_accuracy_ptbdb_rf = grid_search_ptbdb_rf.best_score_
best_parameters_ptbdb_rf = grid_search_ptbdb_rf.best_params_
print("PTBDB dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_ptbdb_rf*100))
print("Best Parameters:", best_parameters_ptbdb_rf)

######XGBoost

In [ ]:
#parameters to search for in xgboost
parameters_xgb = [{'learning_rate': [0.01, 0.1, 1], 
               'loss': ['deviance', 'exponential'],
               'n_estimators': [100,200,400,500],
               'max_depth' : [4,5,6],
               'random_state': [0,43]
               }]

In [ ]:
#mitbih dataset
grid_search_mitbih_xg = GridSearchCV(estimator = xgb_classifier_mitbih,
                           param_grid = parameters_xgb,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_mitbih_xg.fit(X_mitbih_train, Y_mitbih_train)
best_accuracy_mitbih_xg = grid_search_mitbih_xg.best_score_
best_parameters_mitbih_xg = grid_search_mitbih_xg.best_params_
print("mitbih dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_mitbih_xg*100))
print("Best Parameters:", best_parameters_mitbih_xg)

In [ ]:
#ptbdb dataset
grid_search_ptbdb_xg = GridSearchCV(estimator = xgb_classifier_ptbdb,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_ptbdb_xg.fit(X_ptbdb_train, Y_ptbdb_train)
best_accuracy_ptbdb_xg = grid_search_ptbdb_xg.best_score_
best_parameters_ptbdb_xg = grid_search_ptbdb_xg.best_params_
print("PTBDB dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_ptbdb_xg*100))
print("Best Parameters:", best_parameters_ptbdb_xg)

#####LightGBM

In [ ]:
#parameters to search for in lightgbm
parameters_lgbm = [{'boosting_type': ['gbdt','dart','goss'], 
               'learning_rate': [0.01,0.1,1],
               'n_estimators': [100,200,400,500],
               'max_depth' : [4,5,6],
               'random_state': [0,43]
               },
               #in this second model the depth of individual models is left unchanged
               {'boosting_type': ['gbdt','dart','goss'], 
               'learning_rate': [0.01,0.1,1],
               'n_estimators': [100,200,400,500],
               'random_state': [0,43]
               }]

In [ ]:
#mitbih dataset
grid_search_mitbih_lgbm = GridSearchCV(estimator = lgb.LGBMClassifier(),
                           param_grid = parameters_lgbm,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_mitbih_lgbm.fit(X_mitbih_train, Y_mitbih_train)
best_accuracy_mitbih_lgbm = grid_search_mitbih_lgbm.best_score_
best_parameters_mitbih_lgbm = grid_search_mitbih_lgbm.best_params_
print("mitbih dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_mitbih_lgbm*100))
print("Best Parameters:", best_parameters_mitbih_lgbm)

In [ ]:
#ptbdb dataset
grid_search_ptbdb_lgbm = GridSearchCV(estimator = lgb.LGBMClassifier(),
                           param_grid = parameters_lgbm,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search_ptbdb_lgbm.fit(X_ptbdb_train, Y_ptbdb_train)
best_accuracy_ptbdb_lgbm = grid_search_ptbdb_lgbm.best_score_
best_parameters_ptbdb_lgbm = grid_search_ptbdb_lgbm.best_params_
print("PTBDB dataset: ")
print("Best Accuracy: {:.2f} %".format(best_accuracy_ptbdb_lgbm*100))
print("Best Parameters:", best_parameters_ptbdb_lgbm)

###Model Evaluation

#####Random Forest Evaluation

In [ ]:
#mitbih dataset 
Y_mitbih_pred_rf = rf_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_rf, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, rf_classifier_mitbih.predict(X_mitbih_train)))

In [ ]:
#ptbdb dataset
Y_ptbdb_pred_rf = rf_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_rf, target_names = target_names_ptbdb,digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, rf_classifier_ptbdb.predict(X_ptbdb_train)))

#####XGBoost Evaluation

In [ ]:
#mitbih dataset 
Y_mitbih_pred_xg = xgb_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_xg, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, xgb_classifier_mitbih.predict(X_mitbih_train)))


In [ ]:
#ptbdb dataset
Y_ptbdb_pred_xg = xgb_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_xg, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, xgb_classifier_ptbdb.predict(X_ptbdb_train)))

#####LightGBM Evaluation

In [ ]:
#mitbih dataset 
Y_mitbih_pred_lgb = lgb_classifier_mitbih.predict(X_mitbih_test)
target_names_mitbih = ['N', 'S', 'V', 'F', 'Q']
print(classification_report(Y_mitbih_test, Y_mitbih_pred_lgb, target_names = target_names_mitbih, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_mitbih_train, lgb_classifier_mitbih.predict(X_mitbih_train)))

In [ ]:
#ptbdb dataset
Y_ptbdb_pred_lgb = lgb_classifier_ptbdb.predict(X_ptbdb_test)
target_names_ptbdb = ['MI', 'N']
print(classification_report(Y_ptbdb_test, Y_ptbdb_pred_lgb, target_names = target_names_ptbdb, digits = 4))
print("-------------------------------")
print("Training set accuracy: ", accuracy_score(Y_ptbdb_train, lgb_classifier_ptbdb.predict(X_ptbdb_train)))